# Lasso & Ridge Regression
---

# Load packages and data

In [3]:
# Data manipulation
import pandas as pd
import numpy as np

# Regression
from sklearn.model_selection import train_test_split # split data into train and test sets
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # calculate model performance
import statsmodels.api as sm # print model summary
from sklearn.linear_model import Lasso # lasso regression
from sklearn.model_selection import RepeatedKFold # perform cross-validation for tuning the penalty parameter
from sklearn.model_selection import GridSearchCV # perform cross-validation for tuning the penalty parameter
from sklearn.preprocessing import StandardScaler # standardize the data

df = pd.read_csv('/home/simon/Predict_House_Prices/kc_house_data_cleaned_featured.csv', index_col=0)
df.head(5)

id        date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  13-10-2014  221900.0         3       1.00         1180   
1  6414100192  09-12-2014  538000.0         3       2.25         2570   
2  5631500400  25-02-2015  180000.0         2       1.00          770   
3  2487200875  09-12-2014  604000.0         4       3.00         1960   
4  1954400510  18-02-2015  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  yr_built  yr_renovated  zipcode  \
0      5650     1.0           0     0  ...      1955             0    98178   
1      7242     2.0           0     0  ...      1951          1991    98125   
2     10000     1.0           0     0  ...      1933             0    98028   
3      5000     1.0           0     0  ...      1965             0    98136   
4      8080     1.0           0     0  ...      1987             0    98074   

       lat     long  sqft_living15  sqft_lot15  bedrooms_sqft_living  \
0  47.5112 -122.257           1340        5650              0.002542   
1  47.7210 -122.319           1690        7639              0.001167   
2  47.7379 -122.233           2720        8062              0.002597   
3  47.5208 -122.393           1360        5000              0.002041   
4  47.6168 -122.045           1800        7503              0.001786   

   bathrooms_floors  renovated  
0             1.000          0  
1             1.125          1  
2             1.000          0  
3             3.000          0  
4             2.000          0  

[5 rows x 24 columns]

# Lasso Regression

## Prepare data for model

In [5]:
# Create dummy features for categorical variables
df_dummy = pd.get_dummies(df, prefix='Category_', columns=['zipcode'])

# Define dependent variables by dropping unneeded features
X_dummy = df_dummy.drop(columns=['id', 'date', 'price', 'yr_renovated', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'bedrooms_sqft_living', 'bathrooms_floors'], axis=1)

# Define independent variable
y = df['price']

# Add the constant term
X_dummy = sm.add_constant(X_dummy)

# Split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X_dummy, y, test_size=0.25, random_state=1)

## Model tuning

In [ ]:
# List of numerical features
numerical = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
    'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'renovated']

# Standardize numerical features
scaler = StandardScaler().fit(X_train[numerical]) 
X_train[numerical] = scaler.transform(X_train[numerical])
X_test[numerical] = scaler.transform(X_test[numerical])

# Define the model
model = Lasso()

# K-Fold Cross-Validation
cv = RepeatedKFold(n_splits=2, n_repeats=1, random_state=1)

# Tuning grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.1)

# Define the search
search = GridSearchCV(model, grid, scoring=("neg_root_mean_squared_error"), cv=cv, n_jobs=1)

# Perform search
results = search.fit(X_train, y_train)

## Train and test model

In [37]:
# Best shrinkage parameter
alpha = results.best_params_['alpha']
print(results.best_params_)

{'alpha': 0.9}


In [ ]:
# Define tuned model
tunedModel = Lasso(alpha=alpha)
tunedModel.fit(X_train,y_train)

# Making predictions on the test data
y_pred = tunedModel.predict(X_test)

In [39]:
# Calculating the error scores and R² on the test set
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R²:', r2_score(y_test, y_pred))

Mean Absolute Error: 99669.09760855597
Mean Squared Error: 34879675173.32494
Root Mean Squared Error: 186761.0108489589
R²: 0.7821673025781308


- numerical values are standardized to not favor features on a larger scale
- slightly outperforms the OLS linear regression

# Ridge Regression